In [1]:
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm
import math

In [2]:
#file = pd.read_csv('5 yr returns.csv')
file = pd.read_csv('stock returns1.csv')
file

C:\Users\John Flume\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,PERMNO,date,SHRCD,EXCHCD,SICCD,SHRCLS,PRC,RET,SHROUT,sprtrn
0,10000,01/06/1986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001091
1,10000,01/07/1986,10.0,3.0,3990.0,A,-2.56250,C,3680.0,0.014954
2,10000,01/08/1986,10.0,3.0,3990.0,A,-2.50000,-0.024390,3680.0,-0.027268
3,10000,01/09/1986,10.0,3.0,3990.0,A,-2.50000,0.000000,3680.0,-0.008944
4,10000,01/10/1986,10.0,3.0,3990.0,A,-2.50000,0.000000,3680.0,-0.000728
...,...,...,...,...,...,...,...,...,...,...
74074175,93436,12/24/2020,11.0,3.0,9999.0,NaN,661.77002,0.024444,947901.0,0.003537
74074176,93436,12/28/2020,11.0,3.0,9999.0,NaN,663.69000,0.002901,947901.0,0.008723
74074177,93436,12/29/2020,11.0,3.0,9999.0,NaN,665.98999,0.003465,947901.0,-0.002227
74074178,93436,12/30/2020,11.0,3.0,9999.0,NaN,694.78003,0.043229,947901.0,0.001342


In [3]:
# function to clean without reloading data

def clean_returns(file):
    returns = file.copy()
    returns = returns[(returns['SHRCLS'] == 'A') | (returns['SHRCLS'].isna())]
    returns = returns[pd.to_numeric(returns['RET'], errors = 'coerce').notna()]
    returns['RET'] = returns['RET'].apply(lambda x : float(x))
    returns['date'] = pd.to_datetime(returns['date'])
    #making price always positive, using the average spread when there was little to no trading.
    returns['PRC'] = returns['PRC'].apply(lambda x : abs(x))
    returns = returns[pd.to_numeric(returns['SICCD'], errors = 'coerce').notna()]
    returns['SICCD'] = returns['SICCD'].apply(lambda x : int(x))
    returns = returns[~returns['SICCD'].between(6011, 6999)]
    return returns

In [4]:
# creating the returns dataframe
returns = clean_returns(file)

In [5]:
returns['year'] = returns['date'].dt.year
returns['month'] = returns['date'].dt.month
returns['day'] = returns['date'].dt.day

In [6]:
# used as a function to reset for each model instead of pulling data
#distributions and stock split data

def get_split(data):
    split = pd.read_csv(data)
    split = split[split['DISTCD'] == 5523]
    split['DCLRDT'] = pd.to_datetime(split['DCLRDT'])
    split = split[['PERMNO','DISTCD','FACPR','FACSHR','DCLRDT']]
    split = split.dropna()
    split = split[split['DCLRDT'].between('1980-01-01','2020-12-31')]
    # only include big splits
    split = split[split['FACSHR'] >= 1]
    split = split[split['FACPR'] == split['FACSHR']]
    split['year'] = split['DCLRDT'].dt.year
    split['month'] = split['DCLRDT'].dt.month
    split['day'] = split['DCLRDT'].dt.day
    return split

In [7]:
split = get_split('stock distributions.csv')

In [8]:
df = returns

In [9]:
# this retrieves the clean version of ff data
def getff(data):
    ff = pd.read_csv(data)
    ff['date'] = pd.to_datetime(ff['Unnamed: 0'])
    ff = ff[['date','Mkt-RF','SMB','HML','RMW','CMA','RF']]
    ff = ff.set_index('date')
    ff['mkt'] = ff['Mkt-RF'] + ff['RF']
    # make it not in percentage units
    ff = ff/100
    ff = ff.reset_index()
    ff['year'] = ff['date'].dt.year
    ff['month'] = ff['date'].dt.month
    ff['day'] = ff['date'].dt.day
    ff = ff.set_index(keys=['year','month','day'])
    return ff

In [10]:
ff = getff('DailyFF.csv')

## Part 1

In [11]:
# piviot table for returns by firm for specified day
masterreturns = pd.pivot_table(data=returns,columns='PERMNO',index=['year','month','day'],values='RET')

In [12]:
split = split[split.PERMNO.isin(returns.PERMNO.unique())]

In [13]:
# functions to get the returns
def get_two_day_return(PERMNO, year, month, day) -> float:
    return (1+masterreturns.loc[(year,month,day):,PERMNO][:2]).prod() - 1

def get_two_day_return_df(row:pd.Series) -> float:
    return (1+masterreturns.loc[(row['year'], row['month'],row['day']):,row['PERMNO']][:2]).prod() - 1

def get_two_day_return_market(row:pd.Series) ->float:
    return (1+ff.loc[(row['year'], row['month'],row['day']):,'Mkt-RF'][:2]).prod() - 1

In [14]:
# df to store results
splitresults = split.copy()
splitresults['two_day_return'] = splitresults.apply(get_two_day_return_df,axis=1)
splitresults['two_day_mkt']= splitresults.apply(get_two_day_return_market,axis=1)

splitresults['excess_over_market'] = splitresults.two_day_return - splitresults.two_day_mkt

### Average 2 day excess over market returns

In [15]:
Average_return = splitresults[['two_day_return','excess_over_market']].mean()
Average_return

two_day_return        0.030269
excess_over_market    0.030269
dtype: float64

# Part 2

In [16]:
# Returns for splits in the last 6 months

df = returns
df = df[df['PERMNO'].isin(list(split['PERMNO']))]
# old dates are just 26 weeks back from the new
olddates = list(df.date.drop_duplicates() - pd.Timedelta(weeks = 26))
currentdates = list(df.date.drop_duplicates())

equalportfolio2 = pd.DataFrame()
for i in tqdm(range(len(olddates))):
    recentsplit = split[split['DCLRDT'].between(olddates[i], currentdates[i])]
    recentstock = df[df['date']== currentdates[i]]
    recentstock = recentstock[recentstock['PERMNO'].isin(list(recentsplit['PERMNO']))]
    equalportfolio2 = pd.concat([equalportfolio2, recentstock])

# Returns aggregated for the stocks by each date
equalportfolio = equalportfolio2.groupby('date')['RET'].mean()

100%|████████████████████████████████████████████████████████████████████████████| 10340/10340 [17:15<00:00,  9.99it/s]


In [17]:
ff = pd.read_csv('DailyFF.csv')
ff['date'] = pd.to_datetime(ff['Unnamed: 0'], format='%Y%m%d')

ff['year'] = ff['date'].dt.year
ff['month'] = ff['date'].dt.month
ff['day'] = ff['date'].dt.day
ff

,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF,date,year,month,day
0,19630701,-0.67,0.01,-0.35,0.03,0.11,0.012,1963-07-01,1963,7,1
1,19630702,0.79,-0.31,0.24,-0.08,-0.25,0.012,1963-07-02,1963,7,2
2,19630703,0.63,-0.16,-0.09,0.13,-0.24,0.012,1963-07-03,1963,7,3
3,19630705,0.40,0.09,-0.26,0.07,-0.28,0.012,1963-07-05,1963,7,5
4,19630708,-0.63,0.07,-0.19,-0.27,0.06,0.012,1963-07-08,1963,7,8
...,...,...,...,...,...,...,...,...,...,...,...
14680,20211025,0.58,0.37,-0.16,-0.39,-0.35,0.000,2021-10-25,2021,10,25
14681,20211026,0.04,-0.81,-0.32,-0.38,-0.12,0.000,2021-10-26,2021,10,26
14682,20211027,-0.76,-0.95,-1.19,0.86,-0.60,0.000,2021-10-27,2021,10,27
14683,20211028,1.14,0.81,-0.47,-0.30,-0.18,0.000,2021-10-28,2021,10,28


In [18]:
# creating the regressoin data frame using the FF data frame data 
def regression_df(equalportfolio, ff, returntype):
    regression = pd.merge(equalportfolio, ff, how = 'left', left_on = 'date', right_on = 'date')
    regression = regression[[returntype,'Mkt-RF','SMB','HML', 'RF']]
    regression['RF'] = regression['RF']/100
    regression[returntype] = regression[returntype]- regression['RF']
    regression['Mkt-RF'] = regression['Mkt-RF']/100
    regression['SMB'] = regression['SMB']/100
    regression['HML'] = regression['HML']/100
    return regression
regression = regression_df(equalportfolio, ff, 'RET')

## CAPM for equal weighted model

In [19]:
regression

,RET,Mkt-RF,SMB,HML,RF
0,0.036510,-0.0114,0.0019,0.0070,0.00030
1,0.112126,0.0026,0.0019,-0.0001,0.00030
2,0.127300,0.0099,-0.0021,-0.0003,0.00036
3,0.046810,0.0192,-0.0049,-0.0040,0.00036
4,0.035676,-0.0038,0.0059,0.0012,0.00036
...,...,...,...,...,...
10168,0.009032,0.0021,-0.0043,-0.0018,0.00000
10169,0.000005,0.0046,-0.0067,0.0034,0.00000
10170,-0.002154,-0.0040,-0.0142,0.0023,0.00000
10171,0.014180,0.0027,0.0105,0.0006,0.00000


In [20]:
y = regression['RET']
x = regression['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RET   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                 1.661e+04
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        14:16:02   Log-Likelihood:                 32660.
No. Observations:               10173   AIC:                        -6.532e+04
Df Residuals:                   10171   BIC:                        -6.530e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   9.68e-05      7.485      0.000       0.001       0.001
Mkt-RF         1.1280      0.009    128.896      0.000       1.111       1.145
==============================================================================
Omnibus:                     4010.463   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           472390.472
Skew:                           0.892   Prob(JB):                         0.00
Kurtosis:                      36.336   Cond. No.                         90.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
ff_aplha_equal = results.params[0]
ff_t_equal = results.tvalues[0]

In [22]:
ff_aplha_equal

0.0007247385157920975

In [23]:
ff_t_equal

7.48477135580252

## FF 3 factor for equal weighted

In [24]:
y = regression['RET']
x = regression[['Mkt-RF','SMB','HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RET   R-squared:                       0.683
Model:                            OLS   Adj. R-squared:                  0.683
Method:                 Least Squares   F-statistic:                     7293.
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        14:16:02   Log-Likelihood:                 33574.
No. Observations:               10173   AIC:                        -6.714e+04
Df Residuals:                   10169   BIC:                        -6.711e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0008   8.85e-05      8.558      0.000       0.001       0.001
Mkt-RF         1.1300      0.008    140.181      0.000       1.114       1.146
SMB            0.5447      0.015     35.431      0.000       0.515       0.575
HML           -0.4625      0.015    -31.648      0.000      -0.491      -0.434
==============================================================================
Omnibus:                     4661.065   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           837188.277
Skew:                           1.100   Prob(JB):                         0.00
Kurtosis:                      47.387   Cond. No.                         181.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
ff3_aplha_equal = results.params[0]
ff3_t_equal = results.tvalues[0]

In [26]:
ff3_aplha_equal

0.0007576733703088

In [27]:
ff3_t_equal

8.557703766644691

## Sharpe ratio for equal weighted portfolio

In [28]:
sre = ((regression['RET']).mean()/regression['RET'].std()) * math.sqrt(252)
sre

1.11578177198863

In [29]:
returns['mktcap'] = returns['PRC'] * returns['SHROUT']
mktcaps = returns.groupby(['date'])['mktcap'].sum().reset_index()
mktcaps = mktcaps.rename(columns = {'mktcap':'totalcap'})
returns = pd.merge(returns, mktcaps, how = 'left', left_on=['date'], right_on = ['date'])
returns['weigts'] = returns['mktcap']/ returns['totalcap']
returns['weighted_returns'] = returns['weigts'] * returns['RET']

In [30]:
df = returns
df = df[df['PERMNO'].isin(list(split['PERMNO']))]
olddates = list(df.date.drop_duplicates() - pd.Timedelta(weeks = 26))
currentdates = list(df.date.drop_duplicates())
df = df.drop(columns = ['totalcap','weigts','weighted_returns'])

equalportfolio = pd.DataFrame()
for i in tqdm(range(len(olddates))):
    recentsplit = split[split['DCLRDT'].between(olddates[i], currentdates[i])]
    recentstock = df[df['date']== currentdates[i]]
    recentstock = recentstock[recentstock['PERMNO'].isin(list(recentsplit['PERMNO']))]
    equalportfolio = pd.concat([equalportfolio, recentstock])


equalportfolio['mktcap'] = equalportfolio['PRC'] * equalportfolio['SHROUT']

mktcaps = equalportfolio.groupby(['date'])['mktcap'].sum().reset_index()
mktcaps = mktcaps.rename(columns = {'mktcap':'totalcap'})
equalportfolio = pd.merge(equalportfolio, mktcaps, how = 'left', left_on=['date'], right_on = ['date'])
equalportfolio['weigts'] = equalportfolio['mktcap']/ equalportfolio['totalcap']
equalportfolio['weighted_returns'] = equalportfolio['weigts'] * equalportfolio['RET']
avewt = equalportfolio['weigts']

100%|████████████████████████████████████████████████████████████████████████████| 10340/10340 [17:54<00:00,  9.62it/s]


In [31]:
equalportfolio = equalportfolio.groupby('date')['weighted_returns'].sum()
ff = pd.read_csv('DailyFF.csv')
ff['date'] = pd.to_datetime(ff['Unnamed: 0'], format='%Y%m%d')
regression = regression_df(equalportfolio, ff, 'weighted_returns')

In [32]:
ff

,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF,date
0,19630701,-0.67,0.01,-0.35,0.03,0.11,0.012,1963-07-01
1,19630702,0.79,-0.31,0.24,-0.08,-0.25,0.012,1963-07-02
2,19630703,0.63,-0.16,-0.09,0.13,-0.24,0.012,1963-07-03
3,19630705,0.40,0.09,-0.26,0.07,-0.28,0.012,1963-07-05
4,19630708,-0.63,0.07,-0.19,-0.27,0.06,0.012,1963-07-08
...,...,...,...,...,...,...,...,...
14680,20211025,0.58,0.37,-0.16,-0.39,-0.35,0.000,2021-10-25
14681,20211026,0.04,-0.81,-0.32,-0.38,-0.12,0.000,2021-10-26
14682,20211027,-0.76,-0.95,-1.19,0.86,-0.60,0.000,2021-10-27
14683,20211028,1.14,0.81,-0.47,-0.30,-0.18,0.000,2021-10-28


## CAPM for value weighted
This runs a regression to get the CAPM returns
it puts the results below, you should not see any imediatly

In [33]:
y = regression['weighted_returns']
x = regression['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       weighted_returns   R-squared:                       0.624
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                 1.687e+04
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        14:34:38   Log-Likelihood:                 32386.
No. Observations:               10173   AIC:                        -6.477e+04
Df Residuals:                   10171   BIC:                        -6.475e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   9.95e-05      7.509      0.000       0.001       0.001
Mkt-RF         1.1678      0.009    129.896      0.000       1.150       1.185
==============================================================================
Omnibus:                     4499.118   Durbin-Watson:                   1.877
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           319214.482
Skew:                           1.269   Prob(JB):                         0.00
Kurtosis:                      30.325   Cond. No.                         90.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
ff_aplha_value = results.params[0]
ff_t_value = results.tvalues[0]

In [35]:
ff_aplha_value

0.0007469665063487087

In [36]:
ff_t_value

7.508998728829235

## FF 3 Factor with value weighted

In [37]:
y = regression['weighted_returns']
x = regression[['Mkt-RF','SMB','HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       weighted_returns   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     6432.
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        14:34:38   Log-Likelihood:                 32823.
No. Observations:               10173   AIC:                        -6.564e+04
Df Residuals:                   10169   BIC:                        -6.561e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0008   9.53e-05      8.394      0.000       0.001       0.001
Mkt-RF         1.1442      0.009    131.836      0.000       1.127       1.161
SMB            0.0199      0.017      1.202      0.230      -0.013       0.052
HML           -0.4732      0.016    -30.078      0.000      -0.504      -0.442
==============================================================================
Omnibus:                     4945.770   Durbin-Watson:                   1.898
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           392833.838
Skew:                           1.456   Prob(JB):                         0.00
Kurtosis:                      33.303   Cond. No.                         181.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
ff3_aplha_value = results.params[0]
ff3_t_value = results.tvalues[0]

In [39]:
ff3_aplha_value

0.0008001687622576891

In [40]:
ff3_t_value

8.394443532593288

## Sharpe ratio for value weighted Portfolio

In [41]:
srv = regression['weighted_returns'].mean()/(regression['weighted_returns'].std()) * math.sqrt(252)
srv

1.1157780504066588

## Part 3

In [42]:
# Sorted by market caps for each date for top 500 market caps
# Excluded those 500 market caps

top500 = returns.sort_values('mktcap').groupby('date').head(500)
bottom500 = returns.merge(top500, on = list(returns.columns), how = 'left', indicator= True)
returns2 = bottom500[bottom500['_merge'] == 'left_only']

In [43]:
df3  = returns2

## Same as part 2 but excluding top 500 firms

In [44]:
split = get_split('stock distributions.csv')
ff = getff('DailyFF.csv')

In [45]:
masterreturns = pd.pivot_table(data=returns2, columns='PERMNO', index=['year', 'month', 'day'], values='RET')
split = split[split.PERMNO.isin(returns2.PERMNO.unique())]

In [46]:
splitresults = split.copy()
splitresults['two_day_return'] = splitresults.apply(get_two_day_return_df, axis=1)
splitresults['two_day_mkt'] = splitresults.apply(get_two_day_return_market, axis=1)

splitresults['excess_over_market'] = splitresults.two_day_return - splitresults.two_day_mkt
Average_return_WO_500 =  splitresults[['two_day_return', 'excess_over_market']].mean()
Average_return_WO_500

two_day_return        0.029735
excess_over_market    0.029735
dtype: float64

In [47]:
ff = pd.read_csv('DailyFF.csv')
ff['date'] = pd.to_datetime(ff['Unnamed: 0'], format='%Y%m%d')

ff['year'] = ff['date'].dt.year
ff['month'] = ff['date'].dt.month
ff['day'] = ff['date'].dt.day

In [48]:
df = returns2
df = df[df['PERMNO'].isin(list(split['PERMNO']))]
olddates = list(df.date.drop_duplicates() - pd.Timedelta(weeks=26))
currentdates = list(df.date.drop_duplicates())

equalportfolio2 = pd.DataFrame()
for i in tqdm(range(len(olddates))):
    recentsplit = split[split['DCLRDT'].between(olddates[i], currentdates[i])]
    recentstock = df[df['date'] == currentdates[i]]
    recentstock = recentstock[recentstock['PERMNO'].isin(list(recentsplit['PERMNO']))]
    equalportfolio2 = pd.concat([equalportfolio2, recentstock])

equalportfolio = equalportfolio2.groupby('date')['RET'].mean()
ff = pd.read_csv('DailyFF.csv')
ff['date'] = pd.to_datetime(ff['Unnamed: 0'], format='%Y%m%d')

regression = regression_df(equalportfolio, ff, 'RET')

100%|████████████████████████████████████████████████████████████████████████████| 10340/10340 [20:09<00:00,  8.55it/s]


## CAPM on portfolio without 500 biggest equal weighted
This runs a regression to get the CAPM returns
it puts the results below, you should not see any imediatly

In [49]:
y = regression['RET']
x = regression['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RET   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                 1.649e+04
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        15:04:46   Log-Likelihood:                 32554.
No. Observations:               10173   AIC:                        -6.510e+04
Df Residuals:                   10171   BIC:                        -6.509e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   9.78e-05      7.481      0.000       0.001       0.001
Mkt-RF         1.1354      0.009    128.408      0.000       1.118       1.153
==============================================================================
Omnibus:                     4714.229   Durbin-Watson:                   1.914
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           737644.681
Skew:                           1.161   Prob(JB):                         0.00
Kurtosis:                      44.651   Cond. No.                         90.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
ff_wo_500_aplha = results.params[0]
ff_wo_500_t = results.tvalues[0]

In [51]:
ff_wo_500_aplha

0.0007319642200739373

In [52]:
ff_wo_500_t 

7.481230744241065

## FF 3 factors on portfolio without 500 biggest equal weighted

In [53]:
y = regression['RET']
x = regression[['Mkt-RF','SMB','HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RET   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     7229.
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        15:04:47   Log-Likelihood:                 33461.
No. Observations:               10173   AIC:                        -6.691e+04
Df Residuals:                   10169   BIC:                        -6.689e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0008   8.95e-05      8.550      0.000       0.001       0.001
Mkt-RF         1.1373      0.008    139.536      0.000       1.121       1.153
SMB            0.5468      0.016     35.175      0.000       0.516       0.577
HML           -0.4677      0.015    -31.653      0.000      -0.497      -0.439
==============================================================================
Omnibus:                     5511.051   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1313383.655
Skew:                           1.449   Prob(JB):                         0.00
Kurtosis:                      58.589   Cond. No.                         181.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [54]:
ff3_wo_500_aplha_equal = results.params[0]
ff3_wo_500_t_equal = results.tvalues[0]

In [55]:
ff3_wo_500_aplha_equal

0.0007654186965759041

In [56]:
ff3_wo_500_t_equal

8.55019149595779

## Sharp on portfolio without 500 biggest equal weighted

In [57]:
sre500 = regression['RET'].mean()/regression['RET'].std() * math.sqrt(252)
sre500

1.116584539774189

## Average return premium for value weighted portfolio

In [58]:
returns2 = bottom500

In [59]:
df = returns2
df = df[df['PERMNO'].isin(list(split['PERMNO']))]
olddates = list(df.date.drop_duplicates() - pd.Timedelta(weeks = 26))
currentdates = list(df.date.drop_duplicates())
df = df.drop(columns = ['totalcap','weigts','weighted_returns'])

equalportfolio = pd.DataFrame()
for i in tqdm(range(len(olddates))):
    recentsplit = split[split['DCLRDT'].between(olddates[i], currentdates[i])]
    recentstock = df[df['date']== currentdates[i]]
    recentstock = recentstock[recentstock['PERMNO'].isin(list(recentsplit['PERMNO']))]
    equalportfolio = pd.concat([equalportfolio, recentstock])

equalportfolio['mktcap'] = equalportfolio['PRC'] * equalportfolio['SHROUT']
mktcaps = equalportfolio.groupby(['date'])['mktcap'].sum().reset_index()
mktcaps = mktcaps.rename(columns = {'mktcap':'totalcap'})
equalportfolio = pd.merge(equalportfolio, mktcaps, how = 'left', left_on=['date'], right_on = ['date'])
equalportfolio['weigts'] = equalportfolio['mktcap']/ equalportfolio['totalcap']
equalportfolio['weighted_returns'] = equalportfolio['weigts'] * equalportfolio['RET']
avewt = equalportfolio['weigts']

equalportfolio = equalportfolio.groupby('date')['weighted_returns'].sum()
ff = pd.read_csv('DailyFF.csv')
ff['date'] = pd.to_datetime(ff['Unnamed: 0'], format='%Y%m%d')

regression3 = regression_df(equalportfolio, ff, 'weighted_returns')

100%|████████████████████████████████████████████████████████████████████████████| 10340/10340 [18:08<00:00,  9.50it/s]


## CAPM model for value weighted with out 500 largest
This runs a regression to get the CAPM returns
it puts the results below, you should not see any imediatly

In [60]:
y = regression3['weighted_returns']
x = regression3['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       weighted_returns   R-squared:                       0.624
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                 1.687e+04
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        15:23:05   Log-Likelihood:                 32386.
No. Observations:               10173   AIC:                        -6.477e+04
Df Residuals:                   10171   BIC:                        -6.475e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   9.95e-05      7.509      0.000       0.001       0.001
Mkt-RF         1.1678      0.009    129.896      0.000       1.150       1.185
==============================================================================
Omnibus:                     4499.115   Durbin-Watson:                   1.877
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           319213.891
Skew:                           1.269   Prob(JB):                         0.00
Kurtosis:                      30.325   Cond. No.                         90.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
ff_wo_500_aplha_value = results.params[0]
ff_wo_500_t_value = results.tvalues[0]

In [62]:
ff_wo_500_aplha_value

0.0007469679270489195

In [63]:
ff_wo_500_t_value

7.5090113937639735

## FF 3 factor for value weighted without 500 largest
this will get the fama french 3 factor model

In [64]:
y = regression3['weighted_returns']
x = regression3[['Mkt-RF','SMB','HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       weighted_returns   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     6432.
Date:                Thu, 02 Dec 2021   Prob (F-statistic):               0.00
Time:                        15:23:05   Log-Likelihood:                 32823.
No. Observations:               10173   AIC:                        -6.564e+04
Df Residuals:                   10169   BIC:                        -6.561e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0008   9.53e-05      8.394      0.000       0.001       0.001
Mkt-RF         1.1442      0.009    131.836      0.000       1.127       1.161
SMB            0.0199      0.017      1.202      0.230      -0.013       0.052
HML           -0.4732      0.016    -30.078      0.000      -0.504      -0.442
==============================================================================
Omnibus:                     4945.768   Durbin-Watson:                   1.898
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           392833.314
Skew:                           1.456   Prob(JB):                         0.00
Kurtosis:                      33.303   Cond. No.                         181.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
ff3_wo_500_aplha_value = results.params[0]
ff3_wo_500_t_value = results.tvalues[0]

In [66]:
ff3_wo_500_aplha_value

0.0008001702594841583

In [67]:
ff3_wo_500_t_value

8.394458186785158

## Sharpe ratio for value weighted without 500 largest

In [68]:
srv500 = regression3['weighted_returns'].mean()/(regression3['weighted_returns'].std()) * math.sqrt(252)
srv500

1.115779258098591